In [1]:
import sys
import re
sys.path.append('/Users/yuhangzhan/Git/statement_reader')

In [3]:
from src.model.bank_statement import BankStatement
from statement.boa import BankOfAmerica_Credit

In [4]:
fpath = '/Users/yuhangzhan/Git/statement_reader/data/pdf/boa/credit/eStmt_2018-01-28.pdf'
boa = BankOfAmerica_Credit(fpath)

In [5]:
isinstance(boa, BankStatement)

True

In [ ]:
boa.summary

In [ ]:
tuple([value for _, value in boa.transactions['payment'][1].items()]) 

## import conf

In [ ]:
import json

In [ ]:
personal_conf_path = '/Users/yuhangzhan/Git/statement_reader/conf/personal_category_config.json'
default_conf_path = '/Users/yuhangzhan/Git/statement_reader/conf/db_default_config.json'
def import_conf(file_path):
    ''' read configuration json file, return as dict'''
    with open(file_path, 'r') as f:
        raw_conf = ''.join([line for line in f.readlines() if not line.strip().startswith('//')])
        json_conf = json.loads(raw_conf)
    conf = {key:[(int(idx), *[val if val else 'Null' for _, val in fields.items()]) 
                  for idx, fields in value.items() ]
           for key, value in json_conf.items()}
    return conf

In [ ]:
personal_conf = import_conf(personal_conf_path)
default_conf = import_conf(default_conf_path)

In [ ]:
conf = {**default_conf, **personal_conf}

In [ ]:
conf.keys()

## import insert conf sql

In [ ]:
insert_conf_sql_path = '/Users/yuhangzhan/Git/statement_reader/src/database/insert_init_setting.sql'
def import_conf_query(file_path):
    '''read insert conf sql file, return as dict'''
    with open(file_path, 'r') as f:
        raw_sql = ''.join([line for line in f.readlines() if not line.strip().startswith('--') and line.strip()])
    splited_sql = [query for query in raw_sql.split(';') if query]
    dict_sql = [(query.split()[2], query.strip()+';') for query in splited_sql]
    return dict(dict_sql)

In [ ]:
insert_query = import_conf_query(insert_conf_sql_path)

## Connect db

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('/Users/yuhangzhan/Git/statement_reader/data/db/billing.db')
cursor = conn.cursor()

In [ ]:
cursor.executemany("INSERT INTO bank (bank_id, bank_name) values (?, ?);", [(1, 'boa'), (2,'chase')])
conn.commit()

## insert conf

In [ ]:
for table, query in insert_query.items():
    print(table, query, conf[table])

In [ ]:
insert_query

## insert trans

In [ ]:
trans = [tran for _,parts in boa.statement_transaction.items() for tran in parts]

In [ ]:
len(trans)

In [ ]:
def a():
    return tuple([value for _, value in {'types': 'payment',
   'transaction_date': '2018-01-04',
   'posting_date': '2018-01-04',
   'reference_number': 1352,
   'account_number': 7577,
   'amount': -300.0,
   'description': 'Online  from CHK 8'}.items()])
a()

In [ ]:
import sqlite3
from utility.utils import get_project_path
DB_LOC = '/data/db/billing.db'

In [ ]:
db_path = get_project_path('playground.ipynb/..') + DB_LOC
conn = sqlite3.connect(db_path)

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute('select card_number from card')
res = dict(cursor.fetchall())
res

In [ ]:
card_num = set([i['account_number'] for i in trans])

In [ ]:
card_num

## Testing class

In [ ]:
class bank():
    def __init__(self, file_path):
        self.fp = file_path
        self.name = None
    def ha(self):
        print(self.fp + self.name)

class boa_test(bank):
    def __init__(self, file_path):
        super().__init__(file_path)
        self.name = "boa"
    

In [ ]:
b = bank('1')

In [ ]:
t = boa_test('2')

In [ ]:
t.ha()

In [ ]:
boa.raw_pdf_string

In [ ]:
re.findall('Account#[\s\d{4}]+\s(\d{4})', boa.raw_pdf_string)